# Environment set up and Imports

In [ ]:
pip install torch pandas numpy pennylane pennylane-lightning redis matplotlib tqdm scikit-learn

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1rZSIGTcLE5gPt-0nliQTPIZJ0-RtXLU_' -O particolato.csv
!wget 'https://drive.google.com/uc?export=download&id=1z8nOfTyVg1hhpapVRC2egEucIkcQwXl4' -O toy_dataset_features.npy
!wget 'https://drive.google.com/uc?export=download&id=1CYAaNT927vkkoJyimhf-s0_Jr98cdYi6' -O toy_dataset_labels.npy

In [ ]:
!curl -L 'https://drive.google.com/uc?export=download&id=1rZSIGTcLE5gPt-0nliQTPIZJ0-RtXLU_' > particolato.csv
!curl -L 'https://drive.google.com/uc?export=download&id=1z8nOfTyVg1hhpapVRC2egEucIkcQwXl4' > toy_dataset_features.npy
!curl -L 'https://drive.google.com/uc?export=download&id=1CYAaNT927vkkoJyimhf-s0_Jr98cdYi6' -o toy_dataset_labels.npy

In [ ]:
import torch
from typing import Tuple
from torch.utils.data import TensorDataset, random_split, Dataset
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import pennylane as qml
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import redis

# Utility Functions

## Data transforms

In [ ]:
def atan_transform(X: torch.Tensor) -> torch.Tensor:
    return torch.atan(X)

def normalize_transform(X: torch.Tensor) -> torch.Tensor:
    return (X - X.mean(dim=0)) / X.std(dim=0)

def unit_vector_transform(X: torch.Tensor) -> torch.Tensor:
    if len(X.shape) == 1:
        raise ValueError("Unit vector transform requires at least a 2D tensor")
    return X / X.norm(dim=-1).unsqueeze(-1)

def log_transform(X: torch.Tensor) -> torch.Tensor:
    return torch.log(X)

## Dataset retrieval

In [ ]:

PARTITIONS = [0.68, 0.16, 0.16]

def get_sine_datasets(split: bool = False, feature_transformations: list[callable] = [], label_transformations: list[callable] = []) -> Tuple[TensorDataset, TensorDataset, TensorDataset]:
    X = torch.linspace(-4 * torch.pi, 4 * torch.pi, 3000)
    Y = torch.sin(X)
    for transform in feature_transformations:
        X = transform(X)
    X = X.unsqueeze(-1)
    for transform in label_transformations:
        Y = transform(Y)
    dataset = TensorDataset(X, Y)
    return random_split(dataset, PARTITIONS) if split else dataset

def get_toy_classification_datasets(split: bool = False, feature_transformations: list[callable] = [], label_transformations: list[callable] = []):
    X, Y = np.load('toy_dataset_features.npy'), np.load('toy_dataset_labels.npy')
    X, Y = torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)
    for transform in feature_transformations:
        X = transform(X)
    for transform in label_transformations:
        Y = transform(Y)
    dataset = TensorDataset(X, Y)
    return random_split(dataset, PARTITIONS) if split else dataset

def get_pollution_datasets(split: bool = False, feature_transformations: list[callable] = [], label_transformations: list[callable] = []):
    dataframe = pd.read_csv('particolato.csv').drop(columns=['Time'], axis=1).dropna()
    X = torch.tensor(dataframe[["sensors.humidity", "sensors.pm10"]].values, dtype=torch.float32)
    Y = torch.tensor(dataframe["sensors.pm25"].values, dtype=torch.float32)
    for transform in feature_transformations:
        X = transform(X)
    for transform in label_transformations:
        Y = transform(Y)
    dataset = TensorDataset(X, Y)
    return random_split(dataset, PARTITIONS) if split else dataset

In [ ]:
def get_dataloader(datasets: list[Dataset], batch_size: int = 32) -> list[DataLoader]:
    return [DataLoader(dataset, batch_size=batch_size, shuffle=True) for dataset in datasets]

## Data Plotting

In [ ]:
def plot(dataset: TensorDataset, colorbar: bool = False, ax=None):
    X, Y = dataset[:]
    # plot
    if ax is not None:
        ax.set_title('Standard Plot')
        heatmap = ax.scatter(X[:, 0], X[:, 1], c=Y, cmap='viridis')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Label')
        return ax
    else:
        plt.title('PCA')
        heatmap = plt.scatter(X[:, 0], X[:, 1], c=Y, cmap='viridis')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Valore di PM2.5')
        plt.show()

def pca_plot(dataset: TensorDataset, colorbar: bool = False, ax=None):

    X, Y = dataset[:]
    # apply PCA
    pca = PCA(n_components=2)
    X_fitted = pca.fit_transform(X)
    # plot
    if ax is not None:
        ax.set_title('PCA')
        heatmap = ax.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='viridis')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Label')
        return ax
    else:
        plt.title('PCA')
        heatmap = plt.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='viridis')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Valore di PM2.5')
        plt.show()

def lda_plot(dataset: TensorDataset, ax=None):
    X, Y = dataset[:]
    # apply PCA
    lda = LDA(n_components=1)
    X_fitted = lda.fit_transform(X, Y)
    # plot
    if ax is not None:
        ax.set_title('LDA - Nota: è una proiezione 1D, \nla seconda dimensione è fittizia per aiutare la visualizzazione')
        ax.scatter(X_fitted[:, 0], X[:, 0], c=Y, alpha=0.5, cmap='copper')
        ax.hlines(0, X_fitted.min(), X_fitted.max(), color='red', linestyle='--', label='Linea di vera proiezione di LDA')
        ax.legend()
        return ax
    else:
        plt.title('LDA')
        plt.scatter(X_fitted[:, 0], X[:, 0], c=Y, cmap='copper')
        plt.hlines(0, X_fitted.min(), X_fitted.max(), color='red', linestyle='--', label='Linea di vera proiezione di LDA')
        plt.legend()
        plt.show()

def tsne_plot(dataset: TensorDataset, ax=None, colorbar: bool = False):

    X, Y = dataset[:]
    # apply PCA
    tsne = TSNE(n_components=2)
    X_fitted = tsne.fit_transform(X, Y)
    # plot
    if ax is not None:
        ax.set_title('t-SNE')
        heatmap = ax.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='coolwarm')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Label value')
        return ax
    else:
        plt.title('t-SNE')
        heatmap = plt.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='coolwarm')
        if colorbar:
            # add colorbar to the plot in function of Y
            cbar = plt.colorbar(heatmap)
            cbar.set_label('Label value')
        plt.show()

def custom_plot(dataset: TensorDataset, projector_object, ax=None, **projector_args):

    X, Y = dataset[:]
    # apply PCA
    projector = projector_object(**projector_args)
    X_fitted = projector.fit_transform(X, Y)
    # plot
    if ax is not None:
        ax.set_title('Custom')
        ax.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='Spectral')
        return ax
    else:
        plt.title('Custom')
        plt.scatter(X_fitted[:, 0], X_fitted[:, 1], c=Y, cmap='Spectral')
        plt.show()

## Training template

In [ ]:
def training_step(model, optimizer, criterion, compute_accuracy, x, y):
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    accuracy = ((y_pred > 0.5) == y).float().mean() * x.shape[0] if compute_accuracy else -1
    return loss.item() * x.shape[0], accuracy

def training_loop(model, epoch, optimizer, criterion, compute_accuracy, train_loader, epochs):
    model.train()
    traning_loss = 0
    training_accuracy = 0
    for x, y in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', total=len(train_loader), leave=False):
        loss, acc = training_step(model, optimizer, criterion, compute_accuracy, x, y)
        traning_loss += loss
        training_accuracy += acc
    return traning_loss / len(train_loader.dataset), training_accuracy / len(train_loader.dataset)

def validation_step(model, criterion, compute_accuracy, x, y):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    accuracy = ((y_pred > 0.5) == y).float().mean() * x.shape[0] if compute_accuracy else -1
    return loss.item() * x.shape[0], accuracy

@torch.no_grad()
def validation_loop(model, criterion, compute_accuracy, dataloader):
    model.eval()
    validation_loss = 0
    validation_accuracy = 0
    for x, y in tqdm(dataloader, desc='Validation', total=len(dataloader), leave=False):
        loss, acc = validation_step(model, criterion, compute_accuracy, x, y)
        validation_loss += loss
        validation_accuracy += acc
    return validation_loss / len(dataloader.dataset), validation_accuracy / len(dataloader.dataset)

def train_model(model, optimizer, criterion, compute_accuracy, train_loader, val_loader, test_loader, epochs, validation_frequency = 1):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    for epoch in range(epochs):
        train_loss, train_accuracy = training_loop(model, epoch, optimizer, criterion, compute_accuracy, train_loader, epochs)
        if (epoch+1) % validation_frequency == 0:
            val_loss, val_accuracy = validation_loop(model, criterion, compute_accuracy, val_loader)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)
            print(f'Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}' + (f' - Train Accuracy: {train_accuracy:.4f} - Val Accuracy: {val_accuracy:.4f}' if compute_accuracy else ''))
    test_loss, test_accuracy = validation_loop(model, criterion, compute_accuracy, test_loader)
    print(f'Test Loss: {test_loss:.4f}' + (f' - Test Accuracy: {test_accuracy:.4f}' if compute_accuracy else ''))
    return train_losses, val_losses, train_accuracies, val_accuracies, test_loss, test_accuracy

## Training summary plot

In [ ]:
def plot_results(train_losses, val_losses, train_accuracies, val_accuracies, test_loss, test_accuracy, validation_frequency, compute_accuracy):
    x_axis = list(range(0, len(train_losses) * validation_frequency, validation_frequency))
    if compute_accuracy:
        fig, ax = plt.subplots(1, 2, figsize=(16, 6))
        ax[0].plot(x_axis, train_losses, 'b-', label='Train Loss')
        ax[0].plot(x_axis, val_losses, 'r-', label='Val Loss')
        ax[0].hlines(test_loss, 0, x_axis[-1], 'g', label='Test Loss')
        ax[0].set_title('Loss')
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Loss')
        ax[0].legend()


        ax[1].plot(x_axis, train_accuracies, label='Train Accuracy')
        ax[1].plot(x_axis, val_accuracies, label='Val Accuracy')
        ax[1].hlines(test_accuracy, 0, x_axis[-1], 'g', label='Test Accuracy')
        ax[1].set_title('Accuracy')
        ax[1].set_xlabel('Epochs')
        ax[1].set_ylabel('Accuracy')
        ax[1].legend()
        fig.show()
    else:
        fig, ax = plt.subplots(1, 1, figsize=(16, 6))
        ax.plot(x_axis, train_losses, 'b-', label='Train Loss')
        ax.plot(x_axis, val_losses, 'r-', label='Val Loss')
        ax.hlines(test_loss, 0, x_axis[-1], 'g', label='Test Loss')
        ax.set_title('Loss')
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Loss')
        ax.legend()
        fig.show()


## Scoring Utility

In [ ]:
def get_grader():
    r = redis.Redis(
    host='redis-19795.c275.us-east-1-4.ec2.redns.redis-cloud.com',
    port=19795,
    password='HZVxgGIOtp4DR8LeFmKY2LHHa1rPzCRr')
    return r

class ScoreTracker:

    def __init__(self, team_name):
        self.team_name = team_name
        self.redis_client = get_grader()

    def _update_score(self, sset_name, player, points, lt=True):
        self.redis_client.zadd(f'{sset_name}_leaderboard', {player: points}, lt=lt)
        score = self.redis_client.zscore(f'{sset_name}_leaderboard', player)
        self.redis_client.publish(f'{sset_name}_updates', f'{player}: {score}')

    def update_sine_score(self, score):
        self._update_score(sset_name='sine', player=self.team_name, points=score, lt=True)

    def update_classification_score(self, score):
        self._update_score(sset_name='classification', player=self.team_name, points=score, lt=False)

    def update_pm_score(self, score):
        self._update_score(sset_name='pm', player=self.team_name, points=score, lt=True)

# Datasets overview
## Sine Dataset
Just the dataset containing $f(x)=\sin x, \forall x \in \left[-4\pi, 4\pi\right]$

In [ ]:
sine_dataset = get_sine_datasets()
x, sinx = sine_dataset[:]
plt.plot(x, sinx)
plt.show()

## Toy Classification
A dataset generated using the \verb|scikit-learn| utility `make_classification`, where some of the features are redundant. It may help to learn a representation of the data before processing it.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 7))
ds = get_toy_classification_datasets(split=False)
axes[0] = tsne_plot(ds, ax=axes[0])
axes[2] = lda_plot(ds, ax=axes[2])
axes[1] = pca_plot(ds, ax=axes[1])
fig.show()

## Particulate Dataset
This dataset contains samples from the city of Cagliari about the pollution present in the city, the task is to predict the level of small particlate matter (PM2.5) using only the data about humidity and the concentration of large particulate matter (PM10).

In [ ]:
pollution = get_pollution_datasets(label_transformations=[log_transform], split=False)
fig, axes = plt.subplots(1, 3, figsize=(15, 7))
axes[0] = plot(pollution, ax=axes[0], colorbar=True)
axes[1] = pca_plot(pollution, ax=axes[1], colorbar=True)
axes[2] = tsne_plot(pollution, ax=axes[2], colorbar=True)
fig.show()

# Building Quantum Circuit with Pennylane
Pennylane circuit executions revolves around the concept of the Quantun Node (`QNODE`) object, that can be seen as the couple **quantum device** + **circuit function**

## Pennylane devices
The object `qml.device(name=DEVICE_NAME, wires=NUMBER_OF_QUBITS)` defines where the circuit is executed or simulated, and some other settings (e.g. number of shots). Some examples of quantum devices are:
+ `'default.qubit'`: Pennylane default simulator.
+ `'lightning.qubit'`: Pennylane-Lightning simulator, usually faster than `'default.qubit'`
+ `'default.mixed'`: Pennylane noise-enabled simulator.
+ `'qiskit.remote'`: IBM Quantum interface for using real hardware (not installed in this notebook)

In [ ]:
default_device = qml.device('default.qubit', wires=2, shots=2048)
lightning_device = qml.device('lightning.qubit', wires=2, shots=2048)

## Circuit Functions
Circuit functions defines the ordering of quantum gates to be applied to a given device. The circuit for creating the bell state $\lvert \psi> = \frac{\lvert 00>+\lvert 11>}{\sqrt 2}$ is the following.

In [ ]:
def bell_state():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1]) # first index is the controller
  return qml.probs() # return the probability of each state

## Quantum Node
The quantum node is the actual object that can be executed in order to perform the computations, and can be defined in two ways:
+ With the constructor `qml.QNode(CIRCUIT_FUNCTION, QUANTUM_DEVICE)`
+ Using the decorator `@qml.device(QUANTUM_DEVICE)` in the circuit function definition

The circuit associated with a QNode can be plotted using the function `qml.draw` (plain text) and `qml.draw_mpl` (using matplotlib)

In [ ]:
default_qnode = qml.QNode(bell_state, default_device)

@qml.qnode(lightning_device)
def lightning_qnode():
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0, 1])
  return qml.probs()

In [ ]:
print(default_qnode())
print(lightning_qnode())

In [ ]:
print(qml.draw(default_qnode)()) # the syntax is qml.draw(qnode)(qnode_arguments)

In [ ]:
qml.draw_mpl(lightning_qnode)()

## Quantum NN Layer
Pennylane provides the method `qml.qnn.TorchLayer` for creating a `pytorch` compatible layer: it requires two argument, a `QNODE` object that have the parameter `inputs` in its circuit function defintion and a dictionary containing the shape of every other parameters of the circuit, that will be treated as learnable parameter.

For example if we want a 2 circuit that learns an arbitrary pauli rotation and perform a controlled learnable pauli X rotation after performaing the Angle Embedding:

In [ ]:
@qml.qnode(lightning_device, interface='torch') # use torch.Tensor as outputs and inputs
def circuit_function(inputs, rotations, controlled_rotation):
  qml.templates.AngleEmbedding(inputs, wires=[0, 1])
  qml.Rot(*rotations[0], wires=0) # compact version
  qml.Rot(rotations[1, 0], rotations[1, 1], rotations[1, 2], wires=1) # long version
  qml.CRX(controlled_rotation, wires=[0, 1])
  return qml.expval(qml.Z(wires=0)) # retorun the Z expectation value of the first qubit

weight_dict = { 'rotations': (2, 3), 'controlled_rotation': 1 }

qnn = qml.qnn.TorchLayer(circuit_function, weight_dict)
# print(qml.draw(circuit_function)(torch.zeros(2), torch.zeros(weight_dict['rotations']), torch.zeros(weight_dict['controlled_rotation'])))
qml.draw_mpl(circuit_function)(torch.zeros(2), torch.zeros(weight_dict['rotations']), torch.zeros(weight_dict['controlled_rotation']))
plt.show()
qnn(torch.zeros(2))

# Exercises


## Exercise 1: `sine` dataset
The goal is to reduce the Mean Square Error (MSE) as much as possbile

In [ ]:
def get_quantum_node(number_of_qubits: int, device_name: str):
    # pennylane interface with torch requires a circuit function with a mandatory argument named 'inputs', and a sequence of optional
    # arguments that will be the weights of the circuit. TorchLayer requires as argmuments the quantum node associated with the circuit
    # and a dictionary containing the shape of each weight tensor.

    dev = qml.device(device_name, wires=number_of_qubits)

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, weights):

        # data encoding

        # ---- USEFUL TIPS -------------------------
        # qml.templates contains many useful embedding schemes
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own encoding functions
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...


        # variational gates
        # ---- USEFUL TIPS -------------------------
        # qml.templates contains also many useful variational circuits, such as
        # StronglyEntanglingLayers, BasicEntanglerLayers, etc.
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own variational circuits,
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...

        return qml.expval(qml.Z(0))

    return circuit, { 'weights': None } # change here accordingly

    # return circuit, { 'weight_1': None, 'weight_2': None, weight_n: None } # change here
    return circuit, { 'weights': 1 } # change here

class SineRegressor(nn.Module):
    def __init__(self):
        super(SineRegressor, self).__init__()

        # self.preprocess = ... (optional)
        circuit, shapes = get_quantum_node(1, 'default.qubit')
        # circuit = qml.transforms.broadcast_expand(circuit)
        self.quantum_layer = qml.qnn.TorchLayer(circuit, shapes) # a Pennylane TorchLayer
        # self.output_head = ... (optional)

    def forward(self, x):
        # x = self.preprocess(x) (optional)
        x = self.quantum_layer(x)
        # x = self.output_head(x) (optional)
        return x

In [ ]:
team_name = 'team_name'
score_tracker = ScoreTracker(team_name)

In [ ]:
batch_size = None # change
feature_transforms = [] # change
label_transforms = [] # change
train_dataloader, val_dataloader, test_dataloader = get_dataloader(get_sine_datasets(feature_transformations=feature_transforms, label_transformations=label_transforms, split=True), batch_size=batch_size)
model = SineRegressor()
optimizer = optim.SGD(model.parameters(), lr=1e-3) # or Adam, AdamW, etc...
criterion = nn.MSELoss()
compute_accuracy = False
epochs = 10
validation_frequency = max([epochs // 5, 1])
training_results = train_model(model, optimizer, criterion, compute_accuracy, train_dataloader, val_dataloader, test_dataloader, epochs, validation_frequency)
plot_results(*training_results, validation_frequency, compute_accuracy)
# submit score
score_tracker.update_sine_score(training_results[-2])

## Exercise 2: `toy classification` dataset
The goal is to increasae the classification accuracy as much as possbile

In [ ]:
def get_quantum_node(number_of_qubits: int, device_name: str):
    # pennylane interface with torch requires a circuit function with a mandatory argument named 'inputs', and a sequence of optional
    # arguments that will be the weights of the circuit. TorchLayer requires as argmuments the quantum node associated with the circuit
    # and a dictionary containing the shape of each weight tensor.

    dev = qml.device(device_name, wires=number_of_qubits)

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, weights):

        # data encoding

        # ---- USEFUL TIPS -------------------------
        # qml.templates contains many useful embedding schemes
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own encoding functions
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...


        # variational gates
        # ---- USEFUL TIPS -------------------------
        # qml.templates contains also many useful variational circuits, such as
        # StronglyEntanglingLayers, BasicEntanglerLayers, etc.
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own variational circuits,
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...

        return qml.expval(qml.Z(0))

    return circuit, { 'weights': None } # change here accordingly

class QNNClassifier(nn.Module):
    def __init__(self, number_of_qubits: int, local_repetitions: int):
        super(QNNClassifier, self).__init__()

        # self.preprocess = ... (optional)
        circuit, shape = get_quantum_node(number_of_qubits, 'default.qubit', L=local_repetitions)
        # circuit = qml.transforms.broadcast_expand(circuit) # uncomment if you want to use amplitude embedding mid-circuit
        self.quantum_layer = qml.qnn.TorchLayer(circuit, shape) # a Pennylane TorchLayer
        # self.output_head = ... (optional)

    def forward(self, x):
        # x = self.preprocess(x) (optional)
        x = (self.quantum_layer(x) + 1) / 2 # convert output from [-1, 1] to [0, 1]
        # x = self.output_head(x) (optional)
        return x

In [ ]:
team_name = 'team_name'
score_tracker = ScoreTracker(team_name)

In [ ]:
batch_size = None # change
feature_transforms = [] # change
label_transforms = [] # change
train_dataloader, val_dataloader, test_dataloader = get_dataloader(get_toy_classification_datasets(feature_transformations=feature_transforms, label_transformations=label_transforms, split=True), batch_size=batch_size)
model = QNNClassifier(number_of_qubits=3, local_repetitions=3) # define your model
optimizer = optim.SGD(model.parameters(), lr=1e-3) # or Adam, AdamW, etc...
criterion = nn.BCELoss()
compute_accuracy = True
epochs = 20
validation_frequency = max(epochs // 10, 1)
training_results = train_model(model, optimizer, criterion, compute_accuracy, train_dataloader, val_dataloader, test_dataloader, epochs, validation_frequency)
plot_results(*training_results, validation_frequency, compute_accuracy)
# submit score
score_tracker.update_classification_score(training_results[-1].item())

## Exercise 3: `polluttions` dataset
The goal is to reduce the Mean Square Error (MSE) as much as possbile

In [ ]:
def get_quantum_node(number_of_qubits: int, device_name: str):
    # pennylane interface with torch requires a circuit function with a mandatory argument named 'inputs', and a sequence of optional
    # arguments that will be the weights of the circuit. TorchLayer requires as argmuments the quantum node associated with the circuit
    # and a dictionary containing the shape of each weight tensor.

    dev = qml.device(device_name, wires=number_of_qubits)

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, weights):

        # data encoding

        # ---- USEFUL TIPS -------------------------
        # qml.templates contains many useful embedding schemes
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own encoding functions
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...


        # variational gates
        # ---- USEFUL TIPS -------------------------
        # qml.templates contains also many useful variational circuits, such as
        # StronglyEntanglingLayers, BasicEntanglerLayers, etc.
        # https://docs.pennylane.ai/en/stable/introduction/templates.html
        # you can also define your own variational circuits,
        # but to keep it simple try to use only the following gates:
        # Pauli gates/rotations, Hadamard, Controlled-X (CNOT)
        # qml.X(wires=QUBIT_TO_FLIP)
        # qml.RX(ANGLE_OF_ROTATION, wires=QUBIT_TO_ROTATE)
        # qml.Hadamard(wires=QUBIT_TO_APPLY_HADAMARD)
        # qml.CNOT(wires=[CONTROL, TARGET])
        # ...

        return qml.expval(qml.Z(0))

    return circuit, { 'weights': None } # change here accordingly

class PMRegressor(nn.Module):
    def __init__(self, number_of_qubits: int, local_reps: int):
        super(PMRegressor, self).__init__()

        # self.preprocess = ... (optional)
        circuit, shapes = get_quantum_node(number_of_qubits, 'lightning.qubit')
        # circuit = qml.transforms.broadcast_expand(circuit) # uncomment if you want to use amplitude embedding mid-circuit
        self.quantum_layer = qml.qnn.TorchLayer(circuit, shapes) # a Pennylane TorchLayer
        # self.output_head = ... (optional)

    def forward(self, x):
        # x = self.preprocess(x) (optional)
        x = self.quantum_layer(x) * torch.pi / 2
        # x = self.output_head(x) (optional)
        return x

In [ ]:
team_name = 'team_name'
score_tracker = ScoreTracker(team_name)

In [ ]:
batch_size = None # change
feature_transforms = [] # change
label_transforms = [] # change
train_dataloader, val_dataloader, test_dataloader = get_dataloader(get_pollution_datasets(feature_transformations=feature_transforms, label_transformations=label_transforms, split=True), batch_size=batch_size)
model = PMRegressor(...) # None # define your model
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.MSELoss() # MSE Loss
compute_accuracy = False
epochs = 20
validation_frequency = max(epochs // 10, 1)
training_results = train_model(model, optimizer, criterion, compute_accuracy, train_dataloader, val_dataloader, test_dataloader, epochs, validation_frequency)
plot_results(*training_results, validation_frequency, compute_accuracy)

# submit score
score_tracker.update_pm_score(training_results[-2])